In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [2]:
# tools
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [3]:
# prompt
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You're a helpful assistant"), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad', optional=True)]


In [4]:
# llm
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

In [5]:
# memory
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

/var/folders/cg/xkrtyvkn46v_d07jqjh0zkp40000gn/T/ipykernel_17658/131151133.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [6]:
# agent
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [7]:
# invoke the agent
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h5wECBxvSzK0ftOa3AkoV9QX', 'function': {'arguments': '{"x":10.7,"y":7.68}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 146, 'total_tokens': 168, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BGAgJQtm9nTQBhgBnWd7U5QAdgxY7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bba215c1-b144-4886-81ad-c3f6f825bba7-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': 'call_h5wECBxvSzK0ftOa3AkoV9QX', 'type':

In [8]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [9]:
# invoke
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.1759999999999910.7 multiplied by 7.68 is approximately 82.18.

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={})],
 'output': '10.7 multiplied by 7.68 is approximately 82.18.'}

In [10]:
# another execution
agent_executor.invoke({
    "input": "My name is James",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Nice to meet you, James! How can I assist you today?

> Finished chain.


{'input': 'My name is James',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, James! How can I assist you today?'}

In [11]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 9, 'y': 10}`


19.0
Invoking: `multiply` with `{'x': 4, 'y': 2}`


8.0
Invoking: `exponentiate` with `{'x': 2, 'y': 3}`


8.0
Invoking: `subtract` with `{'x': 19, 'y': 8}`


-11.0The result of \( 9 + 10 - 4 \times 2^3 \) is \(-11\).

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-11\\).', additional_kwargs={}, response_metadata={})],
 'output': 'The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-11\\).'}

In [12]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...
Your name is James.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-11\\).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is James.', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is James.'}